In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install --upgrade pandas
!pip install --upgrade torchsummary

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Requirement already up-to-date: pandas in /usr/local/lib/python3.8/dist-packages (1.5.3)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Requirement already up-to-date: torchsummary in /usr/local/lib/python3.8/dist-packages (1.5.1)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import random

from ResNet import Bottleneck, ResNet, ResNet50
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torchvision import models
from torchsummary import summary

In [3]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, out_planes, i_downsample=None, stride=1):
        super(BasicBlock, self).__init__()
        
        # stride를 통해 너비와 높이 조정
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        
        # stride = 1, padding = 1이므로, 너비와 높이는 항시 유지됨
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        
        # x를 그대로 더해주기 위함
        self.shortcut = nn.Sequential()
        self.i_downsample = i_downsample
        
        
        # 만약 size가 안맞아 합연산이 불가하다면, 연산 가능하도록 모양을 맞춰줌
        if stride != 1: # x와 
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_planes)
            )
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x) # 필요에 따라 layer를 Skip
        out = F.relu(out)
        return out
    
def ResNet18(num_classes, channels=3):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes, channels)

def ResNet34(num_classes, channels=3):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes, channels)

In [4]:
def mix_random(col, row, mdsm_body):
    size_suffle = random.randint(0,10)
    switchsource = torch.randint(0, row - 1, (size_suffle,))
    temp = np.zeros((1, col), np.float32)
    
    for i in range(0, int(size_suffle)):
        if i == switchsource[i]:
            continue
        temp = mdsm_body[i, :].copy()
        mdsm_body[i, :] = mdsm_body[switchsource[i], :].copy()
        mdsm_body[switchsource[i], :] = temp.copy()
    return torch.tensor(mdsm_body)

def flip_random(col, row, mdsm_body):
    size_suffle = random.randint(0,10)
    if size_suffle % 2 == 0:
        return torch.tensor(mdsm_body)
    
    int_row = int(row)
    for i in range(0, int(int_row / 2)):
        temp = mdsm_body[i, :].copy()
        mdsm_body[i, :] = mdsm_body[int_row - i - 1, :].copy()
        mdsm_body[int_row - i - 1, :] = temp.copy()
    return torch.tensor(mdsm_body)
    
class MDSMDataset(Dataset):
    def __init__(self, mdsmdata_file):
        self.df = pd.read_csv(mdsmdata_file)
        rating = self.df[['ReviewID', 'reviewStar']]
        self.rating = rating.drop_duplicates('ReviewID')
        self.height = self.df['ReviewID'].value_counts().max()

        mdsm_body = self.df.drop(['reviewNo', 'reviewStar'], axis=1)
        mdsm_body['imageCnt'] = (mdsm_body['imageCnt'] - mdsm_body['imageCnt'].min())/ (mdsm_body['imageCnt'].max() - mdsm_body['imageCnt'].min())
        mdsm_body['helpfulCnt'] = (mdsm_body['helpfulCnt'] - mdsm_body['helpfulCnt'].mean())/ mdsm_body['helpfulCnt'].std()
        body_height, body_width = mdsm_body.shape;
        self.width = body_width - 1

        dummy_mdsd = np.zeros((body_height, self.height, self.width), np.float32)
        mdsm_index = np.zeros(self.rating['ReviewID'].max()+1, int)
        mdsm_count = np.zeros(self.rating['ReviewID'].max()+1, int)
        mdsm_index.fill(-1)

        max_index = int(0)
        for index, body in mdsm_body.iterrows():
            dummy_index = max_index
            if mdsm_index[int(body['ReviewID'])] != -1:
                dummy_index = mdsm_index[int(body['ReviewID'])]
            else:
                mdsm_index[int(body['ReviewID'])] = dummy_index
                max_index = max_index + 1

            dummy_mdsd[dummy_index, mdsm_count[dummy_index]] = body.drop('ReviewID')
            mdsm_count[dummy_index] = mdsm_count[dummy_index] + 1

        self.mdsm_body = dummy_mdsd
            
    def __len__(self):
        return self.rating.shape[0]

    
    def __getitem__(self, idx):
        if trans_stat == True:
            _tensor = flip_random(self.width, self.height, self.mdsm_body[idx])
        else:
            _tensor = torch.tensor(self.mdsm_body[idx])
        rtn_tensor = _tensor.unsqueeze(0)
        return rtn_tensor, self.rating.iloc[idx, 1]

In [5]:
print('-- Loading dataset--')

#dataset = MDSMDataset('amazon_hmdvr_df_tokenized_sentiment_score_extended.csv')
dataset = MDSMDataset('amazon_hmdvr_df_tokenized_sentiment_score_extended_normalized.csv')
train_size = len(dataset) * 0.8
test_size = len(dataset) - train_size

print('-- Building train and test dataset / dataloader--')
train_dataset, test_dataset = random_split(dataset, [int(train_size),int(test_size)])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = 512, shuffle=True, num_workers=0)
#trainloader = torch.utils.data.DataLoader(dataset, batch_size = 256, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size = 512, shuffle=True, num_workers=0)

classes = ['0', '1', '2', '3', '4', '5']

#classes = [0, 1, 2, 3, 4, 5]

#net = ResNet50(6, 1).to('cuda')
#net = ResNet18(6, 1).to('cuda')
net = ResNet34(6, 1).to('cuda')

summary(net, (1, 108, 12))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

-- Loading dataset--
-- Building train and test dataset / dataloader--
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 64, 54, 6]           3,136
       BatchNorm2d-2            [-1, 64, 54, 6]             128
              ReLU-3            [-1, 64, 54, 6]               0
         MaxPool2d-4            [-1, 64, 27, 3]               0
            Conv2d-5            [-1, 64, 27, 3]          36,864
       BatchNorm2d-6            [-1, 64, 27, 3]             128
            Conv2d-7            [-1, 64, 27, 3]          36,864
       BatchNorm2d-8            [-1, 64, 27, 3]             128
        BasicBlock-9            [-1, 64, 27, 3]               0
           Conv2d-10            [-1, 64, 27, 3]          36,864
      BatchNorm2d-11            [-1, 64, 27, 3]             128
           Conv2d-12            [-1, 64, 27, 3]          36,864
      BatchNorm2d-13            

In [6]:
correct = 0
total = 0
EPOCHS = 50
trans_stat = True
print('-- Start training : ', EPOCHS, 'epochs')
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    train_loss = 0
    train_acc = 0
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_loss += loss.item()
        
        pred = outputs.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(labels.data.view_as(pred)).sum()

        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}/{EPOCHS}, {i}](epoch, minibatch): ', f'{running_loss / 100:.5f}')
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
    
    train_loss /= len(trainloader.dataset)
    print('Train Epoch: {} Average loss: {:.4f} Accuracy : {:.4f}%)'.format(epoch, train_loss, 100. * train_acc / len(trainloader.dataset)))

print('Training Done')
trans_stat = False

-- Start training :  50 epochs
Train Epoch: 0 Average loss: 0.0031 Accuracy : 54.0567%)
Train Epoch: 1 Average loss: 0.0025 Accuracy : 58.5532%)
Train Epoch: 2 Average loss: 0.0023 Accuracy : 61.1692%)
Train Epoch: 3 Average loss: 0.0022 Accuracy : 62.6024%)
Train Epoch: 4 Average loss: 0.0022 Accuracy : 62.9512%)
Train Epoch: 5 Average loss: 0.0021 Accuracy : 63.3985%)
Train Epoch: 6 Average loss: 0.0021 Accuracy : 63.2014%)
Train Epoch: 7 Average loss: 0.0021 Accuracy : 62.8374%)
Train Epoch: 8 Average loss: 0.0021 Accuracy : 63.4440%)
Train Epoch: 9 Average loss: 0.0021 Accuracy : 63.5350%)
Train Epoch: 10 Average loss: 0.0021 Accuracy : 63.7019%)
Train Epoch: 11 Average loss: 0.0020 Accuracy : 63.8004%)
Train Epoch: 12 Average loss: 0.0020 Accuracy : 63.5578%)
Train Epoch: 13 Average loss: 0.0020 Accuracy : 63.8232%)
Train Epoch: 14 Average loss: 0.0020 Accuracy : 63.7853%)
Train Epoch: 15 Average loss: 0.0020 Accuracy : 63.8838%)
Train Epoch: 16 Average loss: 0.0020 Accuracy : 64.

In [7]:
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', f'{100*(correct/total):.3f}', '%')

Accuracy on 10,000 test images:  61.632 %


with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        print(outputs.data)
        print(predicted)
        break;